<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ex15notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2022 ex15notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo15.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2022

In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

from statsmodels import tsa
from statsmodels.graphics import tsaplots
from statsmodels.tsa import ar_model
from statsmodels.tsa.arima.model import ARIMA

---
## 演習問題 時系列解析してみよう
---



---
### データの準備


#### データ5: 世帯あたりアイスクリーム・シャーベット購入額



政府統計の総合窓口 e-Stat ( https://www.e-stat.go.jp/ )から入手した，2人以上世帯のアイスクリーム・シャーベットへの支出金額のデータです．2000年1月から2022年10月までの毎月の金額[円]の値から成る時系列です．

このデータは，以下の書籍とウェブページの情報を参考にして作成しました．

- 「時系列解析 ―自己回帰型モデル・状態空間モデル・異常検知―」島田直希，共立出版，2019， https://www.kyoritsu-pub.co.jp/book/b10003204.html
- 「アイスクリームの売れ方」奥村晴彦 https://oku.edu.mie-u.ac.jp/~okumura/stat/160118.html

In [ ]:
# 2000年1月から2022年10月までの毎月のアイスクリーム・シャーベットへの支出金額（世帯あたり）[円]
URL = 'https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/icecream2.csv'
dfIce2 = pd.read_csv(URL, index_col=0)
assert dfIce2.index[0] == '2000-01-01' and dfIce2.index[-1] == '2022-10-01'
dfIce2.index = pd.date_range('2000-01', end='2022-10', freq='MS')
dfIce2

各行に月始の日付が入っていますが，これはデータ作成の都合でそうなっているだけです．各行の値はその月の金額です．

In [ ]:
X_ice = dfIce2['356 アイスクリーム・シャーベット【円】']
fig, ax = plt.subplots(2, 1, figsize=(10, 6))
ax[0].plot(X_ice, '-o')
ax[0].axhline(0, color='gray')
ax[1].plot(X_ice[:'2002'], '-o')
ax[1].axhline(0, color='gray')
plt.tight_layout()
plt.show()

#### `signalA` と `signalB`

In [ ]:
sr = 8000
T = 2
t = np.linspace(0, T, num=T*sr)
signalA = 2 * np.sin(800*np.pi*t) + 0.2*np.random.randn(T*sr)
signalB = 2/3 * np.sin(500*np.pi*t) + 0.2*np.random.randn(T*sr)

---
### 問題1

データ5を見ると，明らかに周期的な変動があります．その周期を調べましょう．

次のセルを実行すると，データ5の時系列の自己相関 $\textrm{Corr}[x_t, x_{t-k}]$ を求めてプロットすることができる．横軸はラグ $k$ を表す．この図を読解して，この時系列の主な季節変動成分の周期は何ヶ月といえるか答えなさい．


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
tsaplots.plot_acf(X_ice, ax=ax)
ax.set_ylim(-1.1, 1.1)
plt.tight_layout()
plt.show()

---
### 問題2

`signalA` と `signalB` という2つの配列に，正弦波信号（にノイズをのせた信号）を標本化して得られる値から成る時系列データが格納されている．いずれも標本化周波数は 8000 [Hz] だった．
次のセルを実行すると，それぞれの自己相関 $\textrm{Corr}[x_t, x_{t-k}]$ を求めてプロットすることができる．横軸はラグ $k$ を表す．
この図から，`signalA`, `signalB` それぞれの元の正弦波信号の周期 [s] と周波数 [Hz] を求めなさい．



In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 8))
tsaplots.plot_acf(signalA, ax=ax[0], title='Autocorrelation of signalA')
ax[0].set_ylim(-1.1, 1.1)
tsaplots.plot_acf(signalB, ax=ax[1], title='Autocorrelation of signalB')
ax[1].set_ylim(-1.1, 1.1)
plt.tight_layout()
plt.show()

次のセルをコメントに従って修正して実行すると，`signalA` と `signalB` の波形を横軸の単位を [s] として描くことができます．その図を読み取れば，上記の問題の正解がわかります．↑の答えを考えたあとで確認しましょう．

ヒント: 横軸の範囲の中にそれぞれの波は何周期分あるだろうか．

In [ ]:
hoge # ←をコメントアウトしてこのセルを実行し，図を読み取れば，上記の正解が分かる...はず

fig, ax = plt.subplots(2, 1, figsize=(10, 8))
ax[0].plot(t, signalA, '-o', label='signalA')
ax[0].axhline(0, color='gray')
ax[0].set_xlabel('t [s]')
ax[0].set_xlim(0, 0.01)
ax[0].set_ylim(-2.5, 2.5)
ax[0].legend()
ax[1].plot(t, signalB, '-o')
ax[1].axhline(0, color='gray', label='signalB')
ax[1].set_xlabel('t [s]')
ax[1].set_xlim(0, 0.02)
ax[1].set_ylim(-2.5, 2.5)
ax[1].legend()
plt.tight_layout()
plt.show()

---
### 問題3

次の2つのコードセルを実行すると，データ5の時系列にARモデルを当てはめる実験が行える．
ARモデルの次数を $6, 9, 12, 24$ 等と変えて実行し，結果を観察しなさい．
コードセルの下の［補足］も参考にしてね．

 

In [ ]:
X_train = X_ice[:'2014-12'] # モデルパラメータの推定に使うデータ
X_test  = X_ice['2015-01':] # 予測の検証用に使うデータ

K = 2 # ARモデルの次数

# X_train を使ってモデルパラメータを推定
model = ar_model.AutoReg(X_train, K, old_names=False)
result = model.fit()
print(result.summary())

In [ ]:
# 得られたモデルを使って予測
X_predicted = result.predict(end='2023-12')

# 結果をプロット
fig, ax = plt.subplots(2, 1, figsize=(10, 10))
XX_predicted = X_predicted[:'2014-12']
ax[0].plot(X_train, '-o', label='true')
ax[0].plot(XX_predicted, '-o', color='red', label=f'predicted (K={K})')
ax[0].axhline(0, color='gray')
ax[0].set_ylim(top=2000)
ax[0].legend()
XX_predicted = X_predicted['2015-01':]
resid = X_test - XX_predicted
ax[1].plot(X_test, '-o', label='true')
ax[1].plot(XX_predicted, '-o', color='red', label=f'predicted (K={K})')
ax[1].axhline(0, color='gray')
ax[1].set_ylim(top=2000)
ax[1].legend()
plt.tight_layout()
plt.show()

［補足］

`AutoReg Model Results` の `Log Likelihood` は，モデルの対数尤度の値を示します．この値を最大化するようパラメータを決めているので，これが大きい方がデータへの当てはまりがよいと言えます．上段のグラフは，モデルパラメータの推定に用いたデータ（ `true` とラベルがついているもの， `X_train`）と，それに対するモデルの予測値（`predicted`，過去 $K$ 時点のデータの値から次の時点の値を予測したもの）を描いたものです．下段のグラフは，2014-12 までの値だけから 2015-01 以降の値を予測したものです．`true` とラベルがついているのがその時期の真の値（`X_test`）です．

- モデル次数を変えると当てはまりの良さはどう変わるだろうか？
- モデル次数を変えると予測の正しさはどう変わるだろうか？


---
### 問題4

次の2つのコードセルを実行すると，データ5の時系列に SARIMA という時系列モデルを当てはめることができる（解説は後述）．1行目をコメントに従って修正して実行すると，AR(12) モデルに切り替えることもできる．両者の結果を比較しなさい．

In [ ]:
isSARIMA = True  # SARIMA モデルを使う (True) / AR(12) を使う (False) か

X_train = X_ice[:'2014-12']
X_test  = X_ice['2015-01':]

if isSARIMA:
    order = (3, 2, 1)
    sorder = (1, 1, 0, 12)
else:
    order = (12, 0, 0)
    sorder = (0, 0, 0, 0)

# X_train を使ってモデルパラメータを推定
model = ARIMA(X_train, order=order, seasonal_order=sorder)
result = model.fit()
print(result.summary())

In [ ]:
# 得られたモデルを使って予測
X_predicted = result.predict(end='2023-12')

# 結果をプロット
if isSARIMA:
    s = 'predicted (SARIMA)'
else:
    s = 'predicted (AR(12))'
fig, ax = plt.subplots(2, 1, figsize=(10, 10))
XX_predicted = X_predicted[:'2014-12']
ax[0].plot(X_train, '-o', label='true')
ax[0].plot(XX_predicted, '-o', color='red', label=s)
ax[0].axhline(0, color='gray')
ax[0].set_ylim(top=2000)
ax[0].legend()
XX_predicted = X_predicted['2015-01':]
resid = X_test - XX_predicted
ax[1].plot(X_test, '-o', label='true')
ax[1].plot(XX_predicted, '-o', color='red', label=s)
ax[1].axhline(0, color='gray')
ax[1].set_ylim(top=2000)
ax[1].legend()
plt.tight_layout()
plt.show()

SARIMA がどのようなものかの真面目な説明はしませんが，ざっくりいうとこんな感じです．

1. AR と移動平均 (Moving Average, MA) モデルというもうひとつのモデルを組み合わせた，ARMA というモデルがある
1. ARMA を発展させた ARIMA というモデルがある
1. ARIMA に季節変動 (Seasonal Variation) を取り入れたものが SARIMA

AR モデルは定常性を仮定したモデルでしたが，SARIMAは平均の変動や季節変動なども考慮したものとなっています．この実験の条件では，SARIMA の方がデータの当てはまりや予測精度が良い結果が得られます．



---
### 新型コロナウイルス感染症関連の時系列データを眺めてみよう

厚生労働省のウェブページ「[データからわかる－新型コロナウイルス感染症情報－](https://covid19.mhlw.go.jp/)」へ行くと，COVID-19 の感染者数，重症者数，死亡者数等のデータを入手することができます（毎日更新）．最新のデータを入手して眺めてみましょう．

#### 人口10万人当たり新規陽性者数

上記ウェブページの「人口10万人当たり新規陽性者数」の項の「オープンデータ」から CSV ファイルを入手できます．

In [ ]:
# 人口10万人当たり新規陽性者数
URL = 'https://covid19.mhlw.go.jp/public/opendata/newly_confirmed_cases_per_100_thousand_population_daily.csv'
df = pd.read_csv(URL, index_col=0, parse_dates=True)
df

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(df['ALL'])
ax.axhline(0, color='gray')
plt.show()

毎日のデータですが，土日等の分の集計が遅れる影響が見られるようです．そこで，7日の幅で移動平均をとってみましょう．

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
for key in ['ALL', 'Tokyo', 'Osaka', 'Kyoto', 'Shiga']:
    X = df[key].rolling(window=7).mean()
    ax.plot(X['2022-01':], label=key)
ax.axhline(0, color='gray')
ax.legend()
plt.show()

ついでに，いくつかの都府県ごとの値も出してみました．「人口10万人当たり」新規陽性者数であることに注意．

#### 入院治療等を要する者等推移

新規陽性者数と同様に「入院治療等を要する者等推移」から CSV ファイルをダウンロードして表示させてみましょう．

In [ ]:
# 入院治療等を要する者等推移
URL = 'https://covid19.mhlw.go.jp/public/opendata/requiring_inpatient_care_etc_daily.csv'
df = pd.read_csv(URL, index_col=0, parse_dates=True)
df

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(df['(ALL) Requiring inpatient care'])
ax.axhline(0, color='gray')
plt.show()

全国の「入院治療等を要する者」の数です．縦軸は $\times 10^6$ つまり100万人単位です．

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
for key in ['Tokyo', 'Osaka', 'Kyoto', 'Shiga']:
    X = df[f'({key}) Requiring inpatient care']
    ax.plot(X, label=key)
ax.axhline(0, color='gray')
ax.legend()
plt.show()

さきほどのデータと違って，こちらの数は人口が大きく影響すること，様々な要因がある（集計法の違いや地域特性の違い，地域によって流行中の変異株が異なり重症化の度合いが異なる場合がある，etc.）ことに注意．ちなみに，東京と大阪の人口比はだいたい 1400万人 : 880万人 = 1 : 0.6 ．

#### 死亡者数の推移



In [ ]:
# 死亡者数の推移
URL = 'https://covid19.mhlw.go.jp/public/opendata/number_of_deaths_daily.csv'
df = pd.read_csv(URL, index_col=0, parse_dates=True)
df

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(df['ALL'])
ax.axhline(0, color='gray')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
for key in ['Tokyo', 'Osaka', 'Kyoto', 'Shiga']:
    X = df[key]
    ax.plot(X, label=key)
ax.axhline(0, color='gray')
ax.legend()
plt.show()

---
### ［よだんだよん］ モデル選択

問題3の実験では，ARモデルの次数を大きくした方がデータへの当てはまりが良くなることを見ました．しかし，次数の大きい，すなわちパラメータの数が多いモデルの方が予測の精度が高いとは限りません．パラメータ数の多い複雑なモデルは，与えられたデータにはよく当てはまる一方で，予測がうまくできないこともよくあります（この辺のことは「機械学習I/II」もっとちゃんと扱います．ちなみに，重回帰分析の回にも似たような話が出てきました（自由度調整済み決定係数））．そのため，いろいろなモデルを作ってみて，それらの中から何らかの規準にもとづいて良さそうなものを選ぶ，という方法がよくとられます．これを「モデル選択」といいます．

モデル選択のために用いられる規準の一つに，AIC（Akaike's Information Criterion, 赤池情報量規準）というものがあります．上の実験結果でも表中に `AIC` として記されています．

参考: Wikipedia の「[赤池情報量規準](https://ja.wikipedia.org/wiki/%E8%B5%A4%E6%B1%A0%E6%83%85%E5%A0%B1%E9%87%8F%E8%A6%8F%E6%BA%96
)」




AR(K)の K をいろいろ変えたものと上記で用いた SARIMA モデルの一つについて，AICの値を見てみましょう．

In [ ]:
X_train = X_ice[:'2014-12']
X_test  = X_ice['2015-01':]

# AR
for K in range(25):
    order = (K, 0, 0)
    sorder = (0, 0, 0, 0)
    model = ARIMA(X_train, order=order, seasonal_order=sorder)
    nparams = len(model.param_names)
    result = model.fit()
    print(f'AR({K}): {nparams}   {result.llf:.2f}   {result.aic:.2f}')

# SARIMA
order = (3, 2, 1)
sorder = (1, 1, 0, 12)
model = ARIMA(X_train, order=order, seasonal_order=sorder)
nparams = len(model.param_names)
result = model.fit()
print(f'SARIMA: {nparams}   {result.llf:.4f}   {result.aic:.4f}')

各行の3つの数値は，左から順に，モデルのパラメータの数，データに対する対数尤度，AICの値，を表します．ARモデルでは，次数 $K$ を大きくすればするほど対数尤度の値が大きくなりデータへの当てはまりが良くなっていますが，AICは $K=13$ で最小となっています．このことから，これらの中では $K=13$ のモデルが予測精度の点では最良であると推測されます．

SARIMAモデルは，この例ではパラメータ数 6 と AR(13) よりも少ないパラメータ数で，データへの当てはまりも AIC もよりよい結果となっています．